In [8]:
import os
import wave
import shutil
from pydub import AudioSegment
import numpy as np

In [1]:
import yaml
from pyannote.database import get_protocol
import os

# read the database config file
with open("/home/rahul/Documents/speaker-change-detection/database.yml", 'r') as stream:
    try:
        database_dict = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

# get the protocol
protocol = get_protocol('AMI.SpeakerDiarization.only_words')

# base directory where all the dirs of files are
base_dir = "/home/rahul/Documents/speaker-change-detection/"

# list of .wav and .rttm file paths
files_list = []

# get the train subset
for current_file in protocol.train():
    # get the uri
    uri = current_file['uri']

    # construct the .wav and .rttm file paths
    wav_file_format = database_dict['Databases']['AMI']
    wav_file = os.path.join(base_dir, wav_file_format.format(uri=uri))

    rttm_file_format = database_dict['Protocols']['AMI']['SpeakerDiarization']['only_words']['train']['annotation']
    rttm_file = os.path.join(base_dir, rttm_file_format.format(uri=uri))

    # append to files_list
    files_list.append([wav_file, rttm_file])

# print the list of .wav and .rttm files
for file_pair in files_list:
    print(f"WAV file: {file_pair[0]}, RTTM file: {file_pair[1]}")

WAV file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/pyannote/amicorpus/IS1000a/audio/IS1000a.Mix-Headset.wav, RTTM file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/only_words/rttms/train/IS1000a.rttm
WAV file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/pyannote/amicorpus/IS1000b/audio/IS1000b.Mix-Headset.wav, RTTM file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/only_words/rttms/train/IS1000b.rttm
WAV file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/pyannote/amicorpus/IS1000c/audio/IS1000c.Mix-Headset.wav, RTTM file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/only_words/rttms/train/IS1000c.rttm
WAV file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/pyannote/amicorpus/IS1000d/audio/IS1000d.Mix-Headset.wav, RTTM file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/only_words/rttms/train/I

In [16]:
# files_list

In [2]:
import os
import wave
import shutil
from pydub import AudioSegment
import numpy as np

def trim_wav_file(input_wav_path, start_time_minutes, end_time_minutes, output_dir):
    start_time_ms = start_time_minutes * 60 * 1000
    end_time_ms = end_time_minutes * 60 * 1000

    audio = AudioSegment.from_wav(input_wav_path)

    trimmed_audio = audio[start_time_ms:end_time_ms]

    trimmed_audio.export(os.path.join(output_dir, 'trimmed_' + os.path.basename(input_wav_path)), format="wav")

def trim_rttm_file(input_rttm_path, start_time_minutes, end_time_minutes, output_dir):
    start_time_seconds = start_time_minutes * 60
    end_time_seconds = end_time_minutes * 60

    with open(input_rttm_path, 'r') as f:
        lines = f.readlines()

    trimmed_lines = [line for line in lines if start_time_seconds <= float(line.split()[3]) <= end_time_seconds]

    with open(os.path.join(output_dir, 'trimmed_' + os.path.basename(input_rttm_path)), 'w') as f:
        for line in trimmed_lines:
            f.write(line)

def trim_files(input_wav_path, input_rttm_path, start_time_minutes, end_time_minutes):
    output_dir = "/home/rahul/Documents/test_data_trimmed"  # specify the output directory here

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # if either wav or rttm file does not exist, return without doing anything
    if not os.path.isfile(input_wav_path) or not os.path.isfile(input_rttm_path):
        print(f"Either .wav file: {input_wav_path} or .rttm file: {input_rttm_path} not found. Skipping.")
        return
    
    trim_wav_file(input_wav_path, start_time_minutes, end_time_minutes, output_dir)
    trim_rttm_file(input_rttm_path, start_time_minutes, end_time_minutes, output_dir)

if __name__ == "__main__":
    start_time_minutes = 0  # specify start time here
    end_time_minutes = 8  # specify end time here
    for wav, rttm in files_list:
        try:
            trim_files(wav, rttm, start_time_minutes, end_time_minutes)
        except:
            continue   

Either .wav file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/pyannote/amicorpus/TS3009b/audio/TS3009b.Mix-Headset.wav or .rttm file: /home/rahul/Documents/speaker-change-detection/AMI-diarization-setup/only_words/rttms/train/TS3009b.rttm not found. Skipping.
